In [3]:
import sys
sys.path.append('../../')

from config.HIVESERVER2_token import *
import pendulum
import jinja2
from pyutil.pysql.sql import SQLClient
from tqdm import trange

# temp = jinja2.Template(open('sql/0.0.1/0.sql','r').read())
# client = SQLClient('ethancao')
# current_timestamp = pendulum.datetime(2023,3,10,0,0,0)
# for i in trange(0, 24 * 15, 6):
#     t = current_timestamp.add(hours = i).to_datetime_string()
#     query = temp.render(current_timestamp = t)
#     for q in query.split(';'):
#         client.execute_trino_query(q)



In [4]:
start_dt, end_dt = '2023-04-14', '2023-04-26'
temp = jinja2.Template(open('sql/0.0.1/1.sql','r').read())
client = SQLClient('ethancao')
query = temp.render(start_dt = start_dt, end_dt = end_dt)
for q in query.split(';'):
    client.execute_trino_query(q)

In [5]:
# temp = jinja2.Template(open('sql/0.0.1/2.sql','r').read())
# client = SQLClient('ethancao')
# query = temp.render(start_dt = start_dt, end_dt = end_dt)
# for q in query.split(';'):
#     client.execute_trino_query(q)

# extraction

In [6]:
import os
import sys

sys.path.append("../../")

import pandas as pd
import pendulum
import tensorflow as tf
from icecream import ic
from jinja2 import Environment, FileSystemLoader
from tqdm import tqdm

from config.HIVESERVER2_token import *
from lmtk.feature_extraction import (
    FEATURE_BIT,
    FEATURE_KEY_SLOT_PREFIX,
    FeatureExtractor,
)
from lmtk.feature_extraction.constants import *
from pyutil.pysql.sql import SQLClient

fe = FeatureExtractor('../../config/dynamic_pricing/0.0.1.conf')
fids = []

In [7]:
import tensorflow as tf
from typing import List

def get_gift_contents(x : bytes):
    return [_x.get("item_goods_id", "") for _x in eval(x.decode())]

def get_bytes_feature(x):
    if not isinstance(x, List):
        x = [x]
    if isinstance(x[0], str):
        x = list(map(lambda y: y.encode(), x))
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=x))

def get_int64_feature(x):
    if not isinstance(x, List):
        x = [x]
    return tf.train.Feature(int64_list = tf.train.Int64List(value=x))

def train_data_to_tfrecord(sample):
    example = tf.train.Example()
    f = example.features.feature

    f['open_id'].CopyFrom(get_bytes_feature(sample.open_id.encode()))
    f['user_id'].CopyFrom(get_bytes_feature(sample.user_id.encode()))

    f['trigger_type'].CopyFrom(get_bytes_feature(sample.trigger_type.encode()))
    f['bundle_price'].CopyFrom(get_int64_feature(sample.bundle_price))
    f['gift_contents'].CopyFrom(get_bytes_feature(sample.gift_contents))
    f['if_buy_pop_gift'].CopyFrom(get_int64_feature(sample.if_buy_pop_gift))

    for k, v in sample.feature.items():
        if k in fe.feature_conf_map:
            f[k].CopyFrom(get_int64_feature(v))

    return example

In [8]:
# pos
q = '''
SELECT *
FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
WHERE grp = 'train'
    AND bundle_price != '99'
    AND if_buy_pop_gift = 1
'''

client = SQLClient('ethancao')
client.execute_trino_query(q)
data = client.to_pandas()

t = data.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
    fids = lambda x: x.feature.apply(lambda x: [_v for _, v in x.items() for _v in v]),
)

fids = fids + [f for fs in t.fids.tolist() for f in fs] 

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.pos.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())

100%|██████████| 77514/77514 [00:52<00:00, 1474.91it/s]


In [9]:
# neg
q = '''
SELECT *
FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
WHERE grp = 'train'
    AND if_buy_pop_gift = 0
    AND bundle_price != '99'
    AND recharge > 5
    AND ((server_id like 'official%' and bundle_price != '499') OR (server_id not like 'official%'))
'''

client = SQLClient('ethancao')
client.execute_trino_query(q)
data = client.to_pandas()


t = data.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
    fids = lambda x: x.feature.apply(lambda x: [_v for _, v in x.items() for _v in v]),
)

fids = fids + [f for fs in t.fids.tolist() for f in fs] 

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.neg.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())

100%|██████████| 655798/655798 [07:03<00:00, 1546.84it/s]


In [10]:
vocab = set(fids)
with open('../../data/vocab/0.0.1.txt', 'w') as f:
    for v in vocab:
        f.write(str(v) +  '\n')

In [11]:
q = '''
SELECT
    pos.*
FROM (
    SELECT 
        b.*
    FROM (
        SELECT open_id, user_id,  MAX(bundle_price) as max_price
        FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
        WHERE if_buy_pop_gift = 1
            and grp = 'train'
            AND bundle_price != '99'
        GROUP BY open_id, user_id
    ) AS a
    INNER JOIN (
        SELECT *
        FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
        WHERE if_buy_pop_gift = 1
            and grp = 'train'
            AND bundle_price != '99'
    ) AS b
    ON a.open_id = b.open_id
        AND a.user_id = b.user_id
        AND a.max_price = b.bundle_price
    
) as pos
INNER JOIN (
    SELECT *
    FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
    WHERE if_buy_pop_gift = 1
        and grp = 'train'
        AND bundle_price != '99'
) as neg
ON pos.open_id = neg.open_id
    AND pos.user_id = neg.user_id
    AND TRY_CAST(pos.bundle_price as int) > TRY_CAST(neg.bundle_price as int)
order by pos.sampleid, neg.sampleid
'''


client = SQLClient('ethancao')
client.execute_trino_query(q)
data1 = client.to_pandas()

t = data1.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
)

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.pos.hard1.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())


client.execute_trino_query(q.replace('pos.*', 'neg.*'))
data2 = client.to_pandas()

t = data2.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
)

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.neg.hard1.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())

 71%|███████▏  | 84237/118128 [00:56<00:22, 1479.77it/s]

In [ ]:
q = '''
SELECT
    pos.*
FROM (
    SELECT *
    FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
    WHERE if_buy_pop_gift = 1
        and grp = 'train'
        AND bundle_price != '99'
) as pos
INNER JOIN (
    SELECT *
    FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
    WHERE if_buy_pop_gift = 0
        and grp = 'train'
        AND bundle_price != '99'
) as neg
ON pos.open_id = neg.open_id
    AND pos.user_id = neg.user_id
    AND TRY_CAST(pos.bundle_price as int) < TRY_CAST(neg.bundle_price as int)
'''


client = SQLClient('ethancao')
client.execute_trino_query(q)
data = client.to_pandas()

t = data.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
)

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.pos.hard2.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())


client.execute_trino_query(q.replace('pos.*', 'neg.*'))
data = client.to_pandas()

t = data.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
)

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.train.neg.hard2.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())

100%|██████████| 61412/61412 [00:37<00:00, 1632.51it/s]


In [ ]:
# valid
q = '''
SELECT *
FROM hive.ml_tmp_db.wgame_trigger_bundle_offline_experiment_0411
WHERE grp = 'valid'
    AND recharge > 0
    AND bundle_price != '99'
'''

client = SQLClient('ethancao')
client.execute_trino_query(q)
data = client.to_pandas()


t = data.dropna().assign(
    bundle_price = lambda x: x.bundle_price.astype(int),
    gift_contents =  lambda x: x.gift_contents.apply(get_gift_contents),
    feature = lambda x : x.feature.apply(fe.extract),
)

with tf.io.TFRecordWriter("../../data/tfrecord/0.0.1.valid.tfrecord") as writer:
    for index in tqdm(t.index):
        example = train_data_to_tfrecord(t.loc[index,])
        writer.write(example.SerializeToString())

100%|██████████| 98105/98105 [01:02<00:00, 1563.81it/s]
